## 🧹 NYPD Motor Vehicle Collisions – Data Cleaning

This notebook performs data cleaning for the NYPD Motor Vehicle Collision dataset (2020–2024), preparing it for exploratory data analysis, visualization, machine learning, and dashboarding.

### 📌 Objectives:
- Understand the structure and quality of the raw dataset
- Remove or fix missing, redundant, or inconsistent data
- Engineer new features (e.g., `year`, `crash_datetime`)
- Prepare a cleaned CSV file for downstream use

### 📁 Input File:
- `data/raw_data/nyc_collisions_2024_to_2020.csv`  

### 📁 Output File:
- `data/cleaned_data/nyc_collisions_cleaned.csv`  
  *(Cleaned and structured dataset ready for analysis)*

### 🔧 Key Cleaning Steps:
1. Load and inspect raw data
2. Analyze null values and drop low-value columns
3. Clean text fields (street names, borough, etc.)
4. Convert and format date/time fields
5. Fill or drop missing data based on importance
6. Save final cleaned dataset

---


### 📦 STEP 1: Import required libraries

In [1]:
import pandas as pd
import numpy as np
import re


### 📂 STEP 2: Load the raw dataset


In [2]:
df = pd.read_csv("../data/raw_data/nyc_collisions_2024_to_2020.csv", low_memory=False)
print("Initial shape:", df.shape)
df.head()

Initial shape: (345642, 29)


,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2024-01-01T00:00:00.000,17:07,BROOKLYN,11207.0,40.665657,-73.888084,"\n, \n(40.665657, -73.888084)",NaN,NaN,779 LIVONIA AVENUE,...,Unspecified,NaN,NaN,NaN,4702082,Sedan,NaN,NaN,NaN,NaN
1,2024-01-01T00:00:00.000,21:56,BROOKLYN,11207.0,40.666430,-73.882835,"\n, \n(40.66643, -73.882835)",NaN,NaN,923 LIVONIA AVENUE,...,Driver Inattention/Distraction,NaN,NaN,NaN,4692079,Sedan,Sedan,NaN,NaN,NaN
2,2024-01-01T00:00:00.000,6:00,QUEENS,11416.0,40.685920,-73.846924,"\n, \n(40.68592, -73.846924)",97 AVENUE,WOODHAVEN BOULEVARD,NaN,...,NaN,NaN,NaN,NaN,4691952,Sedan,NaN,NaN,NaN,NaN
3,2024-01-01T00:00:00.000,5:57,QUEENS,11434.0,40.672382,-73.785740,"\n, \n(40.672382, -73.78574)",BAISLEY BOULEVARD,ROCKAWAY BOULEVARD,NaN,...,Unspecified,NaN,NaN,NaN,4691606,E-Scooter,DELIVERY T,NaN,NaN,NaN
4,2024-01-01T00:00:00.000,2:00,BROOKLYN,11228.0,40.613680,-74.007150,"\n, \n(40.61368, -74.00715)",15 AVENUE,82 STREET,NaN,...,Unspecified,NaN,NaN,NaN,4691924,Sedan,Sedan,NaN,NaN,NaN


### 📊 STEP 3: Check null values and create a summary


In [3]:
# This step gives visibility into data quality, helping you decide which columns to clean, drop, or keep.
null_summary = pd.DataFrame({
    "Null Count": df.isnull().sum(),
    "Null %": (df.isnull().sum() / len(df)) * 100
}).sort_values("Null %", ascending=False)
null_summary


,Null Count,Null %
vehicle_type_code_5,343100,99.264557
contributing_factor_vehicle_5,343011,99.238808
vehicle_type_code_4,337730,97.710926
contributing_factor_vehicle_4,337321,97.592596
vehicle_type_code_3,318124,92.038583
contributing_factor_vehicle_3,316015,91.428414
cross_street_name,210537,60.911868
off_street_name,135223,39.122271
on_street_name,135105,39.088132
vehicle_type_code2,122047,35.310234


### ❌ Step 4: Drop columns with over 90% missing values or low analytical value


In [4]:
columns_to_drop = [
    "vehicle_type_code_5", "contributing_factor_vehicle_5",
    "vehicle_type_code_4", "contributing_factor_vehicle_4",
    "vehicle_type_code_3", "contributing_factor_vehicle_3",
    "cross_street_name", "off_street_name","location"
]
df.drop(columns=columns_to_drop, inplace=True)

#### 🧠 Why These columns were dropped because:

- They had over 90% missing data, which makes them statistically and analytically weak. They represent rare multi-vehicle crash cases (vehicles 3–5).

- cross_street_name and off_street_name were mostly missing and redundant since already on_street_name, borough, and geolocation (latitude, longitude) are there for mapping and grouping. Dropping these reduces noise and improves both model and dashboard performance.

- The location column is a string representation of latitude and longitude. Since already there are separate and more usable latitude and longitude columns, Dropping location can simplify the dataset and avoid redundancy.

### 🧼 Step 5: Clean string fields (standardize text columns)


In [5]:
df["on_street_name"] = df["on_street_name"].fillna("Unknown")

# Ensures consistency in location names and avoids issues with grouping or duplicate entries due to whitespace or punctuation.
text_columns = ["borough", "on_street_name"]

for col in text_columns:
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].apply(lambda x: re.sub(r"[^\w\s]", "", x))

### 📅 Step 6: Convert date columns and extract year


In [6]:
# Enables time-series analysis and time-of-day trends. The year column also allows easy grouping over time.

df["crash_date"] = pd.to_datetime(df["crash_date"], errors='coerce')
df["year"] = df["crash_date"].dt.year

# Combine crash_date + crash_time into crash_datetime
df["crash_datetime"] = pd.to_datetime(
    df["crash_date"].astype(str) + " " + df["crash_time"], errors='coerce'
)

### 🛠 Step 7: Fill or clean key nulls, drop invalid rows


In [ ]:
# Filling important fields keeps useful rows intact
df["zip_code"] = df["zip_code"].fillna("Unknown")
df["contributing_factor_vehicle_1"] = df["contributing_factor_vehicle_1"].fillna("Unspecified")
df["contributing_factor_vehicle_2"] = df["contributing_factor_vehicle_2"].fillna("Unspecified")
df["vehicle_type_code1"] = df["vehicle_type_code1"].fillna("Unknown")
df["vehicle_type_code2"] = df["vehicle_type_code2"].fillna("Unknown")

# Drop rows missing date or location coordinates
# Dropping rows without a date or location ensures valid time-series and map analysis.
df.dropna(subset=["crash_date", "latitude", "longitude"], inplace=True)

### 💾 Step 8: Save cleaned dataset to CSV


In [8]:
df.to_csv("../data/cleaned_data/nyc_collisions_cleaned.csv", index=False)
print("✅ Cleaned data saved. Final shape:", df.shape)

✅ Cleaned data saved. Final shape: (337369, 22)
